In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import nltk
from nltk import ngrams
from nltk.tokenize import word_tokenize
from collections import Counter
import time
nltk.download('punkt')

data_comments_original1  = pd.read_csv('/content/drive/MyDrive/Cognitive_Distortions_DrFarhanAli_Tawseeq/Dataset/teen_sampled_comments_1.csv')
data_comments_original2  = pd.read_csv('/content/drive/MyDrive/Cognitive_Distortions_DrFarhanAli_Tawseeq/Dataset/teen_sampled_comments_2.csv')
data_comments_original3  = pd.read_csv('/content/drive/MyDrive/Cognitive_Distortions_DrFarhanAli_Tawseeq/Dataset/teen_sampled_comments_3.csv', on_bad_lines='skip', engine='python')
data_comments_original4  = pd.read_csv('/content/drive/MyDrive/Cognitive_Distortions_DrFarhanAli_Tawseeq/Dataset/teen_sampled_comments_4.csv', on_bad_lines='skip', engine='python')

data_posts_original  = pd.read_csv('/content/drive/MyDrive/Cognitive_Distortions_DrFarhanAli_Tawseeq/Dataset/teen_sampled_posts.csv')
data_comments_original = pd.concat([data_comments_original1, data_comments_original2, data_comments_original3, data_comments_original4])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Cognitive_Distortions_DrFarhanAli_Tawseeq')
import targetwords

In [ ]:
n_orders = [1, 2, 3, 4, 5]
def generate_ngrams(tokens, n_orders):
    ngram_data = {n: [] for n in n_orders}
    for n in n_orders:
        ngram_data[n] = [' '.join(ngram) for ngram in ngrams(tokens, n)]
    return ngram_data
distortion_ngram_sets = {distortion: set(targetwords.__dict__[distortion]) for distortion in targetwords.__dict__.keys() if 'target_' in distortion}

def process_data_comments(ngram_data_list, original_data):
    results = []
    for i, ngram_data in enumerate(ngram_data_list):
        distortion_counts = {distortion: 0 for distortion in distortion_ngram_sets.keys()}
        for n in n_orders:
            for ngram_str in ngram_data[n]:
                for distortion in distortion_counts.keys():
                    if ngram_str in distortion_ngram_sets[distortion]:
                        distortion_counts[distortion] += 1
        row = {'Comment No.': original_data.index[i]}
        row.update(distortion_counts)
        results.append(row)
    return pd.DataFrame(results)

def process_data_posts(ngram_data_list_title, ngram_data_list_body, original_data):
    results = []
    for i in range(len(ngram_data_list_title)):
        distortion_counts = {distortion: 0 for distortion in distortion_ngram_sets.keys()}
        
        for n in n_orders:
            for ngram_str in ngram_data_list_title[i][n]:
                for distortion in distortion_counts.keys():
                    if ngram_str in distortion_ngram_sets[distortion]:
                        distortion_counts[distortion] += 1
        
        for n in n_orders:
            for ngram_str in ngram_data_list_body[i][n]:
                for distortion in distortion_counts.keys():
                    if ngram_str in distortion_ngram_sets[distortion]:
                        distortion_counts[distortion] += 1
        row = {'Post No.': original_data.index[i]}
        row.update(distortion_counts)
        results.append(row)
    return pd.DataFrame(results)

In [ ]:
#tokenizing
tokenized_data_comments = [word_tokenize(str(text).lower()) for text in data_comments_original['body']]
all_ngrams = [generate_ngrams(tokens, n_orders) for tokens in tokenized_data_comments]
final_results = process_data_comments(all_ngrams, data_comments_original)
final_results.to_csv('/content/drive/MyDrive/Cognitive_Distortions_DrFarhanAli_Tawseeq/Teenagers/EXCEL/teen_all_comments.csv', index=False)

tokenized_title = [word_tokenize(str(text).lower()) for text in data_posts_original['title']]
tokenized_body = [word_tokenize(str(text).lower()) for text in data_posts_original['body']]
all_ngrams_title = [generate_ngrams(tokens, n_orders) for tokens in tokenized_title]
all_ngrams_body = [generate_ngrams(tokens, n_orders) for tokens in tokenized_body]
final_results_posts = process_data_posts(all_ngrams_title, all_ngrams_body, data_posts_original)

final_results_posts.to_csv('your_path', index=False)

In [ ]:
covid_start = 1579478400  
covid_end = 1594598400    
end_date = 1697372672

before_covid = data_comments_original[data_comments_original['created_utc'] < covid_start]
during_covid = data_comments_original[(data_comments_original['created_utc'] >= covid_start) & (data_comments_original['created_utc'] <= covid_end)]
after_covid = data_comments_original[(data_comments_original['created_utc'] > covid_end) & (data_comments_original['created_utc'] <= end_date)]

tokenized_comments_before_covid_body = [word_tokenize(str(text).lower()) for text in before_covid['body']]
tokenized_comments_during_covid_body = [word_tokenize(str(text).lower()) for text in during_covid['body']]
tokenized_comments_after_covid_body = [word_tokenize(str(text).lower()) for text in after_covid['body']]

all_ngrams_before = [generate_ngrams(tokens, n_orders) for tokens in tokenized_comments_before_covid_body]
all_ngrams_during = [generate_ngrams(tokens, n_orders) for tokens in tokenized_comments_during_covid_body]
all_ngrams_after = [generate_ngrams(tokens, n_orders) for tokens in tokenized_comments_after_covid_body]

final_results_comments_before = process_data_comments(all_ngrams_before, before_covid)
final_results_comments_during = process_data_comments(all_ngrams_during, during_covid)
final_results_comments_after = process_data_comments(all_ngrams_after, after_covid)

final_results_comments_before.to_csv('your_path', index=False)
final_results_comments_during.to_csv('your_path', index=False)
final_results_comments_after.to_csv('your_path', index=False)

In [ ]:
covid_start = 1579478400  
covid_end = 1594598400    
end_date = 1697372672

before_covid = data_posts_original[data_posts_original['created_utc'] < covid_start]
during_covid = data_posts_original[(data_posts_original['created_utc'] >= covid_start) & (data_posts_original['created_utc'] <= covid_end)]
after_covid = data_posts_original[(data_posts_original['created_utc'] > covid_end) & (data_posts_original['created_utc'] <= end_date)]

tokenized_posts_before_covid_body = [word_tokenize(str(text).lower()) for text in before_covid['body']]
tokenized_posts_during_covid_body = [word_tokenize(str(text).lower()) for text in during_covid['body']]
tokenized_posts_after_covid_body = [word_tokenize(str(text).lower()) for text in after_covid['body']]

tokenized_posts_before_covid_title = [word_tokenize(str(text).lower()) for text in before_covid['title']]
tokenized_posts_during_covid_body = [word_tokenize(str(text).lower()) for text in during_covid['title']]
tokenized_posts_after_covid_body = [word_tokenize(str(text).lower()) for text in after_covid['title']]

all_ngrams_posts_before_body = [generate_ngrams(tokens, n_orders) for tokens in tokenized_posts_before_covid_body]
all_ngrams_posts_during_body = [generate_ngrams(tokens, n_orders) for tokens in tokenized_posts_during_covid_body]
all_ngrams_posts_after_body = [generate_ngrams(tokens, n_orders) for tokens in tokenized_posts_after_covid_body]

all_ngrams_posts_before_title = [generate_ngrams(tokens, n_orders) for tokens in tokenized_posts_before_covid_title]
all_ngrams_posts_during_title = [generate_ngrams(tokens, n_orders) for tokens in tokenized_posts_during_covid_body]
all_ngrams_posts_after_title = [generate_ngrams(tokens, n_orders) for tokens in tokenized_posts_after_covid_body]

final_results_posts_before = process_data_posts(all_ngrams_posts_before_title, all_ngrams_posts_before_body, before_covid)
final_results_posts_during = process_data_posts(all_ngrams_posts_during_title, all_ngrams_posts_during_body, during_covid)
final_results_posts_after = process_data_posts(all_ngrams_posts_after_title, all_ngrams_posts_after_body, after_covid)

final_results_posts_before.to_csv('your_path', index=False)
final_results_posts_during.to_csv('your_path', index=False)
final_results_posts_after.to_csv('your_path', index=False)
